In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Detection and Facial Landmarks models
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

def get_landmark_from_image(image):
    '''This function inputs an image and returns all landmarks with their indices in the image'''
    
    # Convert the frame to RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    landmarks = face_mesh.process(rgb_image)
    iw, ih, _ = image.shape

    if landmarks.multi_face_landmarks:
        for face_landmarks in landmarks.multi_face_landmarks:
            global frame_data 
            frame_data = []
            for i, landmark in enumerate(face_landmarks.landmark):
                x, y = landmark.x * iw, landmark.y * ih
                frame_data.append((x, y))

        return np.array(frame_data, dtype=np.float32)
    else:
        # Return an empty list if no faces are detected
        return np.array([])

image = cv2.imread("SampleImage/image_10.jpg")
print(get_landmark_from_image(image))

[[236.97577 363.3197 ]
 [235.39589 332.10156]
 [235.71912 342.55432]
 [231.14932 299.7956 ]
 [235.3026  321.95142]
 [235.32323 308.84427]
 [235.44658 277.8462 ]
 [201.22597 276.93784]
 [235.33514 256.50143]
 [235.18875 244.17117]
 [234.88828 198.8655 ]
 [237.03906 367.7904 ]
 [237.07129 372.88293]
 [237.1063  376.5002 ]
 [237.10635 376.63043]
 [237.16663 381.36127]
 [237.18297 387.12152]
 [237.18854 393.34894]
 [236.57622 402.52917]
 [235.54909 337.4231 ]
 [230.88234 337.6308 ]
 [183.74757 246.84258]
 [214.34163 285.25543]
 [209.60628 286.0239 ]
 [205.02885 285.81573]
 [199.19403 281.1035 ]
 [218.38773 283.02872]
 [207.28258 259.5592 ]
 [212.73639 260.40784]
 [202.22539 261.3161 ]
 [199.042   265.0854 ]
 [195.12946 288.37906]
 [218.40677 420.0193 ]
 [199.54895 274.91528]
 [181.72656 288.18143]
 [189.76239 282.9062 ]
 [211.72484 326.821  ]
 [230.62292 361.52228]
 [231.58191 373.18555]
 [224.16203 366.87134]
 [220.17532 372.40463]
 [226.7768  374.68732]
 [223.03047 376.83264]
 [214.48805

##### 1. Get the difference between the two distances from 2 images (abnormal/normal)

In [3]:
def distance(keypoint,keypoints):
    n = len(keypoints)
    np_1 = np.array([keypoint]*n)
    np_2 = np.array(keypoints)
    return ((np_1[:,0]-np_2[:,0])**2 + (np_1[:,1]-np_2[:,1])**2)**(1/2)

def calculate_distance(keypoints):
    return np.array([distance(i,keypoints) for i in keypoints])

def Diff_2_image(img1,img2):
    kp1 = get_landmark_from_image(img1)
    kp2 = get_landmark_from_image(img2)
    d_kp1 = calculate_distance(kp1)
    d_kp2 = calculate_distance(kp2)
    return d_kp1-d_kp2

img_normal = cv2.imread("SampleImage/image_10.jpg")
img_abnormal = cv2.imread("SampleImage/image_31.jpg")
print("Differ")
result = Diff_2_image(img_normal,img_abnormal)
print(result)

Differ
[[ 0.         -2.8458939  -1.583313   ... -1.0904694  -0.8978729
  -0.92056274]
 [-2.8458939   0.         -1.5071115  ...  0.5563507   0.6167526
   0.39855194]
 [-1.583313   -1.5071115   0.         ...  0.66265106  0.7689209
   0.848259  ]
 ...
 [-1.0904694   0.5563507   0.66265106 ...  0.          0.07983017
  -0.16211319]
 [-0.8978729   0.6167526   0.7689209  ...  0.07983017  0.
   0.77216816]
 [-0.92056274  0.39855194  0.848259   ... -0.16211319  0.77216816
   0.        ]]


In [4]:
import pandas as pd

# Assuming 'result' is your DataFrame
df = pd.DataFrame(result)

# Set custom column names
custom_column_names = list(range(0, 478))  # Replace with your actual column names
df.columns = custom_column_names

# Set custom row names
custom_row_names = list(range(0, 478)) # Replace with your desired row names
df.index = custom_row_names
df.index.name = "Index"
# Now, your DataFrame has custom row names and custom column names.

# Save the DataFrame to a CSV file
df.to_csv('test.csv', index=True, header=True)

In [5]:
df = pd.read_csv("test.csv")
abs(df)

,Index,0,1,2,3,4,5,6,7,8,...,468,469,470,471,472,473,474,475,476,477
0,0,0.000000,2.845894,1.583313,0.361053,2.535400,1.557804,0.358444,9.327049,0.509819,...,6.781471,5.275398,8.252602,8.548439,5.525604,1.176003,1.436668,1.090469,0.897873,0.920563
1,1,2.845894,0.000000,1.507111,2.606978,0.306178,1.224340,2.970409,13.852688,3.127640,...,10.452499,8.670624,11.826454,12.542088,9.333935,0.348160,0.073135,0.556351,0.616753,0.398552
2,2,1.583313,1.507111,0.000000,1.145725,1.094671,0.017250,1.945587,11.115787,2.096039,...,8.219933,6.647369,9.697678,10.092613,6.969402,0.594749,0.425583,0.662651,0.768921,0.848259
3,3,0.361053,2.606978,1.145725,0.000000,2.430405,1.780250,0.384380,11.664005,0.594688,...,7.436655,5.520569,8.874073,9.720755,6.249470,0.472084,0.192135,0.511803,0.881798,0.248768
4,4,2.535400,0.306178,1.094671,2.430405,0.000000,0.882762,2.581661,14.317898,2.758217,...,10.526527,8.619083,11.844612,12.753139,9.487759,0.351006,0.574364,0.094196,0.141541,0.391197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,473,1.176003,0.348160,0.594749,0.472084,0.351006,0.891571,1.122253,8.320103,2.981981,...,3.436684,1.368389,4.634708,5.734070,2.696354,0.000000,0.693443,0.112675,1.070406,0.275173
474,474,1.436668,0.073135,0.425583,0.192135,0.574364,0.916870,1.755772,9.021900,3.817658,...,4.133900,2.062088,5.317768,6.434685,3.375179,0.693443,0.000000,0.952955,1.763634,0.351895
475,475,1.090469,0.556351,0.662651,0.511803,0.094196,0.680569,0.625385,7.081574,2.699142,...,2.281391,0.327732,3.265556,4.470844,1.761551,0.112675,0.952955,0.000000,0.079830,0.162113
476,476,0.897873,0.616753,0.768921,0.881798,0.141541,0.903542,0.167084,7.252010,1.779356,...,2.368958,0.305424,3.571819,4.662064,1.669697,1.070406,1.763634,0.079830,0.000000,0.772168


In [6]:
def longest_path_max_sum(matrix):

    # matrix = np.array([[0, 2, 1],
    #                 [2, 0, 6],
    #                 [1, 6, 0]])

    num_rows, num_cols = matrix.shape

    # Create a table to store the maximum sum values
    dp_table = np.zeros((num_rows, num_cols))

    # Initialize the first cell of the table
    dp_table[0, 0] = matrix[0, 0]

    # Fill in the dp_table using dynamic programming
    for i in range(num_rows):
        for j in range(num_cols):
            # Skip the first cell (0,0) since we initialized it already
            if i == 0 and j == 0:
                continue
            from_top = dp_table[i - 1, j] if i > 0 else 0
            from_left = dp_table[i, j - 1] if j > 0 else 0
            dp_table[i, j] = max(from_top, from_left) + matrix[i, j]

    # The value in the bottom-right cell of the dp_table contains the maximum sum
    max_sum = dp_table[num_rows - 1, num_cols - 1]

    # Trace back the path from (2,2) to (0,0)
    i, j = num_rows - 1, num_cols - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        from_top = dp_table[i - 1, j] if i > 0 else float('-inf')
        from_left = dp_table[i, j - 1] if j > 0 else float('-inf')
        if from_top > from_left:
            i -= 1
        else:
            j -= 1
        path.append((i, j))

    # Reverse the path to get it from (0,0) to (2,2)
    path = path[::-1]

    print("Longest path with maximum sum:", path)
    print("Maximum sum:", max_sum)

longest_path_max_sum(abs(result))

Longest path with maximum sum: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (34, 2), (34, 3), (34, 4), (34, 5), (34, 6), (34, 7), (34, 8), (34, 9), (34, 10), (34, 11), (34, 12), (34, 13), (34, 14), (34, 15), (34, 16), (34, 17), (34, 18), (34, 19), (34, 20), (35, 20), (36, 20), (36, 21), (37, 21), (38, 21), (39, 21), (40, 21), (41, 21), (42, 21), (43, 21), (44, 21), (44, 22), (44, 23), (44, 24), (44, 25), (44, 26), (44, 27), (44, 28), (44, 29), (44, 30), (44, 31), (44, 32), (44, 33), (44, 34), (45, 34), (46, 34), (47, 34), (48, 34), (49, 34), (50, 34), (51, 34), (51, 35), (51, 36), (52, 36), (52, 37), (53, 37), (54, 37), (55, 37), (56, 37), (57, 37), (58, 37), (58, 38), (58, 39), (58, 40), (58, 41), (58, 42), (58, 43), (58

##### 2. Get the landmarks from all images in folder 

In [7]:
import os

def process_images_in_folder(folder_path):
    
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif'))]

    landmark_arrays = []
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path)

        landmarks = get_landmark_from_image(image)
        if landmarks is not None:
            landmark_arrays.append(landmarks)
        else:
            landmark_arrays.append([0])
    return landmark_arrays

folder_path = 'DataSets\\Origin\\Normal'
landmark_arrays_normal = process_images_in_folder(folder_path)
folder_path = 'DataSets\\Origin\\Abnormal'
landmark_arrays_abnormal = process_images_in_folder(folder_path)
print("Number of normal pictures", len(landmark_arrays_normal))
print("Number of abnormal pictures", landmark_arrays_abnormal[10].shape)


Number of normal pictures 400
Number of abnormal pictures (478, 2)


#### 2.1 Calculate mean distance for each class -> Difference 

In [11]:
def mean_distance_all_image(landmark_array):
    distance_matrices = []
    for landmark in landmark_array:
        if landmark.shape == (0, ):
            continue
        distances = calculate_distance(landmark)
        distance_matrices.append(distances)
    distance_matrices = np.mean(np.array(distance_matrices), axis = 0)
    return distance_matrices

distance_matrices_normal = mean_distance_all_image(landmark_arrays_normal)
distance_matrices_abnormal = mean_distance_all_image(landmark_arrays_abnormal)

In [9]:
result_matrix_1 = distance_matrices_normal - distance_matrices_abnormal
result_matrix_1

# # Assuming 'result' is your DataFrame
# df = pd.DataFrame(result_matrix_1)

# # Set custom column names
# custom_column_names = list(range(0, 478))  # Replace with your actual column names
# df.columns = custom_column_names

# # Set custom row names
# custom_row_names = list(range(0, 478)) # Replace with your desired row names
# df.index = custom_row_names
# df.index.name = "Index"
# # Now, your DataFrame has custom row names and custom column names.

# # Save the DataFrame to a CSV file
# df.to_csv('test1.csv', index=True, header=True)

array([[ 0.       ,  2.7990723,  3.453827 , ..., 19.719414 , 17.802933 ,
        16.707756 ],
       [ 2.7990723,  0.       , -1.3162766, ..., 15.641308 , 13.492744 ,
        12.317123 ],
       [ 3.453827 , -1.3162766,  0.       , ..., 16.377182 , 14.381817 ,
        13.423176 ],
       ...,
       [19.719414 , 15.641308 , 16.377182 , ...,  0.       ,  2.0509791,
         3.0127487],
       [17.802933 , 13.492744 , 14.381817 , ...,  2.0509791,  0.       ,
         1.9200196],
       [16.707756 , 12.317123 , 13.423176 , ...,  3.0127487,  1.9200196,
         0.       ]], dtype=float32)

##### 2.2 Calculate difference distances for each image -> mean difference 

In [10]:
def mean_distance_all_image(landmark_array):
    distance_matrices = []
    for landmark in landmark_array:
        distances = calculate_distance(landmark)
        distance_matrices.append(distances)
    return distance_matrices

distance_matrices_normal = mean_distance_all_image(landmark_arrays_normal)
distance_matrices_abnormal = mean_distance_all_image(landmark_arrays_abnormal)

result_matrices = []
for matrix_abnormal, matrix_normal in zip(distance_matrices_abnormal, distance_matrices_normal):
    if matrix_abnormal.shape == (0, ) or matrix_normal.shape == (0, ):
        continue
    result_matrix = np.array(matrix_normal) - np.array(matrix_abnormal)
    result_matrices.append(result_matrix)

if result_matrices:
    mean_result_matrix = np.mean(result_matrices, axis=0)
    print("Mean Result Matrix:")
    print(mean_result_matrix)
else:
    print("No valid matrices to calculate the mean.")

# # Assuming 'result' is your DataFrame
# df = pd.DataFrame(mean_result_matrix)

# # Set custom column names
# custom_column_names = list(range(0, 478))  # Replace with your actual column names
# df.columns = custom_column_names

# # Set custom row names
# custom_row_names = list(range(0, 478)) # Replace with your desired row names
# df.index = custom_row_names
# df.index.name = "Index"
# # Now, your DataFrame has custom row names and custom column names.

# # Save the DataFrame to a CSV file
# df.to_csv('test2.csv', index=True, header=True)


Mean Result Matrix:
[[ 0.         2.8078144  3.480915  ... 19.828833  17.947071  16.765587 ]
 [ 2.8078144  0.        -1.3329772 ... 15.6068325 13.495889  12.191251 ]
 [ 3.480915  -1.3329772  0.        ... 16.453358  14.493439  13.440946 ]
 ...
 [19.828833  15.6068325 16.453358  ...  0.         2.0199757  3.0502808]
 [17.947071  13.495889  14.493439  ...  2.0199757  0.         1.9540793]
 [16.765587  12.191251  13.440946  ...  3.0502808  1.9540793  0.       ]]


#### Find the biggest change in all distance

In [12]:
import numpy as np

def longest_path_max_sum(matrix):

    num_rows, num_cols = matrix.shape

    # Create a table to store the maximum sum values
    dp_table = np.zeros((num_rows, num_cols))

    # Initialize the first cell of the table
    dp_table[0, 0] = matrix[0, 0]

    # Fill in the dp_table using dynamic programming
    for i in range(num_rows):
        for j in range(num_cols):
            # Skip the first cell (0,0) since we initialized it already
            if i == 0 and j == 0:
                continue
            from_top = dp_table[i - 1, j] if i > 0 else 0
            from_left = dp_table[i, j - 1] if j > 0 else 0
            dp_table[i, j] = max(from_top, from_left) + matrix[i, j]

    # The value in the bottom-right cell of the dp_table contains the maximum sum
    max_sum = dp_table[num_rows - 1, num_cols - 1]

    # Trace back the path from (num_rows-1, num_cols-1) to (0,0)
    i, j = num_rows - 1, num_cols - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        from_top = dp_table[i - 1, j] if i > 0 else float('-inf')
        from_left = dp_table[i, j - 1] if j > 0 else float('-inf')
        if from_top > from_left:
            i -= 1
        else:
            j -= 1
        path.append((i, j))

    # Reverse the path to get it from (0,0) to (num_rows-1, num_cols-1)
    path = path[::-1]

    # Extract and sort the values along the longest path in decreasing order
    values_on_longest_path = [(matrix[i, j], (i, j)) for i, j in path]
    values_on_longest_path.sort(reverse=True)

    print("Longest path with maximum sum:", path)
    print("Maximum sum:", max_sum)
    print("Values along the longest path (sorted in decreasing order):")
    for value, (i, j) in values_on_longest_path:
        print(f"Value: {value}, Index: ({i}, {j})")

longest_path_max_sum(abs(result_matrix_1))

Longest path with maximum sum: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (11, 7), (11, 8), (11, 9), (11, 10), (12, 10), (13, 10), (14, 10), (15, 10), (16, 10), (17, 10), (18, 10), (19, 10), (20, 10), (21, 10), (22, 10), (23, 10), (24, 10), (25, 10), (26, 10), (27, 10), (28, 10), (29, 10), (30, 10), (31, 10), (32, 10), (33, 10), (34, 10), (35, 10), (36, 10), (37, 10), (38, 10), (39, 10), (40, 10), (41, 10), (42, 10), (43, 10), (44, 10), (45, 10), (46, 10), (47, 10), (48, 10), (49, 10), (50, 10), (51, 10), (52, 10), (53, 10), (54, 10), (55, 10), (56, 10), (57, 10), (58, 10), (59, 10), (60, 10), (61, 10), (62, 10), (63, 10), (64, 10), (65, 10), (66, 10), (67, 10), (68, 10), (69, 10), (70, 10), (71, 10), (72, 10), (73, 10), (74, 10), (75, 10), (76, 10), (77, 10), (78, 10), (79, 10), (80, 10), (81, 10), (82, 10), (83, 10), (84, 10), (85, 10), (86, 10), (87, 10), (88, 10), (89, 10),

In [65]:
set([171, 10,175, 10,199, 10,208, 10,148, 10,152, 10,332, 171,396, 338,140, 10,332, 148,332, 208,428, 338,
332, 175,176, 10,332, 140,201, 10,200, 10,332, 176,32, 10,332, 199])

{10, 32, 140, 148, 152, 171, 175, 176, 199, 200, 201, 208, 332, 338, 396, 428}

In [64]:
print(len([10, 32, 140, 148, 152, 171, 175, 176, 199, 200, 201, 208, 332, 338, 396, 421, 428]))


17
